In [1]:
from dolfinx import geometry, fem, mesh, plot, io
from mpi4py import MPI
from petsc4py import PETSc
from petsc4py.PETSc import ScalarType, ComplexType
from ufl import (TrialFunction, Measure, TestFunction, dx, ds, grad, div, inner, lhs, rhs)

import timeit
import numpy as np
import pandas as pd

from utils.dolfinx import BoundaryCondition, generate_boundary_measure, project, L2_norm, H1_norm
from utils.plotting import Mpl1DAnimator

from IPython.display import Video
from pathlib import Path

In [2]:
save_dir = "./results/1dh-wq"
Path(save_dir).mkdir(parents=True, exist_ok=True)

In [3]:
def generate_boundaries(points):
    return [(1, lambda x: np.isclose(x[0], points[0])), 
            (2, lambda x: np.isclose(x[0], points[1]))]

In [4]:
def problem_setup(N, points, p, T, t, ct, dt, fluid, сontaminant, contamination_level, solver, pc):
    """
    Performs problem configuration w.r.t. given parameters
    """
    
    # Mesh and function space definition
    domain = mesh.create_interval(MPI.COMM_WORLD, N, points)
    V = fem.FunctionSpace(domain, ("CG", 1))

    u = TrialFunction(V)
    v = TestFunction(V)
    
    # Definition of density and speed functions
    w_ro, w_c, w_eta = fluids.loc[fluid, ['Density', 'Speed of sound', 'Viscosity']]
    c_ro, c_c, c_eta = fluids.loc[сontaminant, ['Density', 'Speed of sound', 'Viscosity']]
    
    ro, c, eta = [(1.0-contamination_level)*vals[0] + contamination_level*vals[1] 
                  for vals in [(w_ro, c_ro), (w_c, c_c), (w_eta, c_eta)]]
        
    # Construction of bilinear form and linear functional
    GAMMA, BETA = 0.5, 0.5
    
    mm = ScalarType(ro) * inner(u, v) * dx
    aa = ScalarType(ro * c**2) * inner(grad(u), grad(v)) * dx
    cc = ScalarType(4./3 * eta) * inner(grad(u), grad(v)) * dx
    
    F = mm + (dt * GAMMA * cc) + (0.5 * dt * dt * BETA * aa)
        
    boundaries = generate_boundaries(points)
    measure = generate_boundary_measure(boundaries, domain)
    
    u_D = lambda x: x[0] * 0.0
    u_N1 = fem.Constant(domain, ScalarType(-p))
    u_N2 = fem.Constant(domain, ScalarType(0.0))
    
    bcs = [BoundaryCondition("Dirichlet", 1, u_D, V, u, v, measure).bc]
    nbcs = [BoundaryCondition("Neumann", 2, u_N1, V, u, v, measure).bc,
            BoundaryCondition("Neumann", 2, u_N2, V, u, v, measure).bc] 
    
    return {
        'Params': (N, points, fluid, ro, c, T, dt, t, ct, GAMMA, BETA, solver, pc),
        'FunctionSpace': (domain, V, u, v),
        'Forms': (mm, aa, cc), 
        'Problem': (F, bcs, nbcs)
    }

In [5]:
def solve_problem(config):
    N, points, fluid, ro, c, T, dt, t_stop, ct, GAMMA, BETA, solver_type, pc = config['Params']
    domain, V, u, v = config['FunctionSpace']
    mm, aa, cc = config['Forms']
    F, bcs, nbcs = config['Problem']
    
    # Create initial condition
    initial_condition = lambda x: x[0] * 0.0
    
    uj = fem.Function(V)
    uj_d = fem.Function(V)
    uj_dd = fem.Function(V)
    uj_d_d_tg = fem.Function(V)
    
    uj.interpolate(initial_condition)
    uj_d.interpolate(initial_condition)
    uj_dd.interpolate(initial_condition)
    uj_d_d_tg.interpolate(initial_condition)
    
    p = project(ro * c**2 * uj.dx(0), domain, ('CG', 1))
    p.x.array[:] = ro * c**2 * uj_d.x.array
    
    # Construct the left and right hand side of the problem
    bilinear_form = fem.form(lhs(F))
    
    A = fem.petsc.assemble_matrix(bilinear_form, bcs=bcs)
    A.assemble()
    
    solver = PETSc.KSP().create(domain.comm)
    solver.setOperators(A)
    solver.setType(solver_type.lower())
    solver.getPC().setType(pc.lower())
    
    # Solve problem at each time step,
    num_steps = int(T / dt)
    t = 0.0
    
    time_data = {
        't'   : [0.0,],
        'uj'  : [np.copy(uj.x.array[:]),],
        'uj_d': [np.copy(uj_d.x.array[:]),],
        'p'   : [np.copy(p.x.array[:]),]
    }
    
    uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
    L1 = nbcs[0] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    L2 = nbcs[1] + (cc * uj_d) + (aa * uj) + (aa * uj_d_d_tg)
    linear_form_1 = fem.form(rhs(L1))
    linear_form_2 = fem.form(rhs(L2))
    
    b1 = fem.petsc.create_vector(linear_form_1)
    b2 = fem.petsc.create_vector(linear_form_2)
    
    has_norm = False
    L2_n, H1_n = 0.0, 0.0
    
    for i in range(num_steps):
        t += dt
        
        uj_d_d_tg.x.array[:] = dt * GAMMA * uj_d.x.array
        
        b = b1 if t < t_stop else b2
        linear_form = linear_form_1 if t < t_stop else linear_form_2

        with b.localForm() as loc_b:
            loc_b.set(0)
        fem.petsc.assemble_vector(b, linear_form)

        # Apply Dirichlet boundary condition to the vector
        fem.petsc.apply_lifting(b, [bilinear_form], [bcs])
        b.ghostUpdate(addv=PETSc.InsertMode.INSERT, mode=PETSc.ScatterMode.FORWARD)
        fem.petsc.set_bc(b, bcs)
        
        # Solve linear problem
        solver.solve(b, uj_dd.vector)
        uj_dd.x.scatter_forward()
        
        # Update solution
        uj.x.array[:] = uj.x.array + dt * uj_d.x.array + 0.5 * dt**2 * uj_dd.x.array
        uj_d.x.array[:] = uj_d.x.array + dt * uj_dd.x.array
        
        if i % 5 == 0:
            p = project(ro * c**2 * uj.dx(0), domain, ('CG', 1))
            
            time_data['t'].append(t)
            time_data['uj'].append(np.copy(uj.x.array))
            time_data['uj_d'].append(np.copy(uj_d.x.array))
            time_data['p'].append(np.copy(p.x.array))
        
        if not has_norm and t >= ct:
            L2_n = L2_norm(uj)
            H1_n = H1_norm(uj)

            has_norm = True

    return L2_n, H1_n, uj, uj_d, p, time_data

#### Available fluids

In [6]:
fluids = pd.read_csv('../data/physical_properties.csv', sep=';', index_col='Fluid')
fluids

,Density,Speed of sound,Viscosity,Heat,Thermal conductivity,Thermal expansion
Fluid,,,,,,
Water,1000,1500,0.000894,4182.0,0.598,0.00015
Fuel oil,890,1360,2.022000,2090.0,0.150,0.00070
Oil,760,1470,0.000500,NaN,NaN,NaN
Glycerin,1260,1905,1.500000,NaN,NaN,NaN


In [7]:
config_1 = problem_setup(
    N=1000,
    points=[0.0, 0.02],
    p=1.0,
    T=5.0e-5,
    t=1.5e-6,
    ct=2.0e-5,
    dt=5.0e-8,
    fluid='Water',
    сontaminant='Fuel oil',
    contamination_level=0.0,
    solver='CG',
    pc='ASM'
)
config_2 = problem_setup(
    N=1000,
    points=[0.0, 0.02],
    p=1.0,
    T=5.0e-5,
    t=1.5e-6,
    ct=2.0e-5,
    dt=5.0e-8,
    fluid='Water',
    сontaminant='Fuel oil',
    contamination_level=0.35,
    solver='CG',
    pc='ASM'
)

In [8]:
L2_n_1, H1_n_1, uj_1, uj_d_1, p_1, time_data_1 = solve_problem(config_1)
L2_n_2, H1_n_2, uj_2, uj_d_2, p_2, time_data_2 = solve_problem(config_2)
N, points, *_ = config_1['Params']

In [9]:
def data_diff(data_1, data_2):
    return list(arr_1-arr_2 for arr_1, arr_2 in zip(data_1, data_2))

In [10]:
mpa = Mpl1DAnimator(layout=[['1', '2', '3'], 
                            ['4', '5', '6'], 
                            ['7', '8', '9']], time=time_data_1['t'])
mpa.update_figure(figsize=(16, 10), fontsize=20)

mpa.add_fun(fun=uj_1, time_data=time_data_1['uj'], type='real', 
            points=points, N=N, axes_id='1', color='r', title='Displacement(O)')
mpa.add_fun(fun=uj_2, time_data=time_data_2['uj'], type='real', 
            points=points, N=N, axes_id='2', color='r', title='Displacement(C)')
mpa.add_fun(fun=uj_1, time_data=data_diff(time_data_1['uj'], time_data_2['uj']), type='real', 
            points=points, N=N, axes_id='3', color='r', title='Displacement(D)')

mpa.add_fun(fun=uj_d_1, time_data=time_data_1['uj_d'], type='real', 
            points=points, N=N, axes_id='4', color='g', title='Velocity(O)')
mpa.add_fun(fun=uj_d_2, time_data=time_data_2['uj_d'], type='real', 
            points=points, N=N, axes_id='5', color='g', title='Velocity(C)')
mpa.add_fun(fun=uj_d_1, time_data=data_diff(time_data_1['uj_d'], time_data_2['uj_d']), type='real', 
            points=points, N=N, axes_id='6', color='g', title='Velocity(D)')

mpa.add_fun(fun=p_1, time_data=time_data_1['p'], type='real', 
            points=points, N=N, axes_id='7',  color='b', title='Pressure(O)')
mpa.add_fun(fun=p_2, time_data=time_data_2['p'], type='real', 
            points=points, N=N, axes_id='8',  color='b', title='Pressure(C)')
mpa.add_fun(fun=p_1, time_data=data_diff(time_data_1['p'], time_data_2['p']), type='real', 
            points=points, N=N, axes_id='9',  color='b', title='Pressure(D)')

mpa.write(path=save_dir, filename='animation.mp4', fps=10, interval=1000)

In [11]:
Video(f'{save_dir}/animation.mp4', width=800)

In [30]:
def calc_convergence():
    results = list()
    
    for idx, N in enumerate([100 * 2**i for i in range(10)]):
        config_1 = problem_setup(
            N=N,
            points=[0.0, 0.02],
            p=1.0,
            T=5.0e-5,
            t=1.5e-6,
            ct=2.0e-5,
            dt=5.0e-8,
            fluid='Water',
            сontaminant='Fuel oil',
            contamination_level=0.0,
            solver='CG',
            pc='ASM'
        )
        config_2 = problem_setup(
            N=N,
            points=[0.0, 0.02],
            p=1.0,
            T=5.0e-5,
            t=1.5e-6,
            ct=2.0e-5,
            dt=5.0e-8,
            fluid='Water',
            сontaminant='Fuel oil',
            contamination_level=0.35,
            solver='CG',
            pc='ASM'
        )
        
        L2_n_1, H1_n_1, uj_1, _, _, time_data_1 = solve_problem(config_1)
        L2_n_2, H1_n_2, uj_2, _, _, time_data_2 = solve_problem(config_2)
        
        t_idx = 81
        uj_1.x.array[:] = time_data_1['uj'][t_idx] - time_data_2['uj'][t_idx]
        
        L2_n_3 = L2_norm(uj_1)
        H1_n_3 = H1_norm(uj_1)
        
        results.append([N, L2_n_1, L2_n_2, L2_n_3, H1_n_1, H1_n_2, H1_n_3, np.nan, np.nan])
        
        if len(results) > 2:
            # rl = np.log(results[-1][3]/results[-2][3])/np.log(N/results[-2][0])
            # rh = np.log(results[-1][6]/results[-2][6])/np.log(N/results[-2][0])
            rl = np.log(abs(results[-3][3] - results[-2][3])/abs(results[-2][3] - results[-1][3]))
            rh = np.log(abs(results[-3][6] - results[-2][6])/abs(results[-2][6] - results[-1][6]))
            
            results[-1][-2] = rl
            results[-1][-1] = rh
        
        print(f'Step {idx+1}')
        print(results[-1])
        print()
    
    return results

In [31]:
cdata = calc_convergence()
header = ['N', '||u(O)||L2', '||u(C)||L2', '||u(D)||L2', 
               '||u(O)||H1', '||u(C)||H1', '||u(D)||H1', 'r_L2', 'r_H1']
cdf = pd.DataFrame(cdata, columns=header).set_index('N')
cdf

Step 1
[100, 1.0368472402730364e-13, 1.1628560522089843e-13, 2.2499353940012327e-14, 2.108135018584886e-11, 2.2269716576791663e-11, 1.9638376839867216e-11, nan, nan]

Step 2
[200, 1.0368204845724236e-13, 1.1628293002150417e-13, 2.2488518646544945e-14, 2.1090605325848563e-11, 2.2235305304646787e-11, 1.956881830487016e-11, nan, nan]

Step 3
[400, 1.0368137840893745e-13, 1.1628226515449385e-13, 2.2485817026308082e-14, 2.1089031756602895e-11, 2.222760319828225e-11, 1.9537730947596325e-11, 1.3889570389947514, 0.8053674102594103]

Step 4
[800, 1.0368120740560149e-13, 1.162820995591284e-13, 2.2485228223380883e-14, 2.1070180306088856e-11, 2.2226345784831567e-11, 1.9547111755852285e-11, 1.523515420870716, 1.1981352906892808]

Step 5
[1600, 1.0368118515879674e-13, 1.1628205727502454e-13, 2.248495802342623e-14, 2.1099360009542555e-11, 2.2224647266722782e-11, 1.951180526730584e-11, 0.7789292811739439, -1.3254008312433325]

Step 6
[3200, 1.0368117617376692e-13, 1.1628204724476324e-13, 2.24849826088

,||u(O)||L2,||u(C)||L2,||u(D)||L2,||u(O)||H1,||u(C)||H1,||u(D)||H1,r_L2,r_H1
N,,,,,,,,
100,1.036847e-13,1.162856e-13,2.249935e-14,2.108135e-11,2.226972e-11,1.963838e-11,NaN,NaN
200,1.036820e-13,1.162829e-13,2.248852e-14,2.109061e-11,2.223531e-11,1.956882e-11,NaN,NaN
400,1.036814e-13,1.162823e-13,2.248582e-14,2.108903e-11,2.222760e-11,1.953773e-11,1.388957,0.805367
800,1.036812e-13,1.162821e-13,2.248523e-14,2.107018e-11,2.222635e-11,1.954711e-11,1.523515,1.198135
1600,1.036812e-13,1.162821e-13,2.248496e-14,2.109936e-11,2.222465e-11,1.951181e-11,0.778929,-1.325401
3200,1.036812e-13,1.162820e-13,2.248498e-14,2.111032e-11,2.222507e-11,1.954139e-11,2.397009,0.176745
6400,1.036812e-13,1.162820e-13,2.248499e-14,2.110587e-11,2.222520e-11,1.954885e-11,0.835022,1.378199
12800,1.036812e-13,1.162820e-13,2.248500e-14,2.110327e-11,2.222523e-11,1.954939e-11,1.415012,2.618550
25600,1.036812e-13,1.162820e-13,2.248500e-14,2.110248e-11,2.222523e-11,1.954939e-11,1.403243,5.210311
